In [22]:
#血統のデータをスクレイプする 特徴量を作成する 当日も取れる情報がよい
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import time
from tqdm import tqdm as tqdm
import numpy as np
from sklearn.metrics import roc_curve, roc_auc_score
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from urllib.request import urlopen
pd.set_option('display.max_columns', 100)

In [23]:
#払戻金をスクレイプ(testデータに対して, するのが望ましい)
def sclape_return_tables(race_id_list, pre_return_tables={}):
    return_tables = pre_return_tables
    for race_id in tqdm(race_id_list):
        if race_id in return_tables.keys():
            continue
        try:
            url = 'https://db.netkeiba.com/race/' + race_id
            f = urlopen(url)
            html = f.read()
            html = html.replace(b'<br />', b'br')
            dfs = pd.read_html(html)
            return_tables[race_id] = pd.concat([dfs[1], dfs[2]])
            time.sleep(0.1)
        except IndexError:
            continue
        except Exception as e:
            # for文から抜け出す
            print('Error',e)
            break

    #辞書形から, データフレーム形に変換
    for key in return_tables.keys():
        return_tables[key].index = [key] * len(return_tables[key])

    return_tables = pd.concat([return_tables[key] for key in return_tables.keys()], sort=False)
    return return_tables

In [28]:
race_results = pd.read_pickle('1-2results2015_2020.pickle')
horse_results = pd.read_pickle('1-4horse_results2015_2020.pickle')

In [29]:
race_id_list = race_results.index.unique()
race_id_list

Index(['201503010101', '201503010102', '201503010103', '201503010104',
       '201503010105', '201503010106', '201503010107', '201503010108',
       '201503010109', '201503010110',
       ...
       '202001020204', '202001020205', '202001020206', '202001020207',
       '202001020208', '202001020209', '202001020210', '202001020211',
       '202001020212', '202001020301'],
      dtype='object', length=15752)

In [30]:
return_tables = sclape_return_tables(race_id_list)

100%|██████████| 15752/15752 [44:43<00:00,  5.87it/s]  


In [31]:
return_tables.index.unique()

Index(['201503010101', '201503010102', '201503010103', '201503010104',
       '201503010105', '201503010106', '201503010107', '201503010108',
       '201503010109', '201503010110',
       ...
       '202001020204', '202001020205', '202001020206', '202001020207',
       '202001020208', '202001020209', '202001020210', '202001020211',
       '202001020212', '202001020301'],
      dtype='object', length=15752)

In [32]:
return_tables.head()

,0,1,2,3
201503010101,単勝,13,1160,5
201503010101,複勝,13br12br5,220br110br460,4br1br8
201503010101,枠連,7 - 7,910,4
201503010101,馬連,12 - 13,930,3
201503010101,ワイド,12 - 13br5 - 13br5 - 12,"430br3,900br730",3br33br7


In [33]:
return_tables.to_pickle('1-12return_tables2015_2020.pickle')